In [1]:
import os
import urllib.request
import re
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
if not os.path.exists("../data/the-verdict.txt"):
    url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt"
    file_path = "../data/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [3]:
with open("../data/the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

In [4]:
raw_text

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)\n\n"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it\'s going to send the value of my picture \'way up; but I don\'t think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing\'s lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn\'s "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?\n\nWell!--even 

In [5]:
len(raw_text)

20479

In [6]:
text = "Hello, my name is Hrishi. I like coding."
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [word for word in preprocessed if word.strip()]
len(preprocessed)

4690

In [7]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [8]:
vocab = {token:integer for integer,token in enumerate(all_words)}
vocab

{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '--': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'Ah': 12,
 'Among': 13,
 'And': 14,
 'Are': 15,
 'Arrt': 16,
 'As': 17,
 'At': 18,
 'Be': 19,
 'Begin': 20,
 'Burlington': 21,
 'But': 22,
 'By': 23,
 'Carlo': 24,
 'Chicago': 25,
 'Claude': 26,
 'Come': 27,
 'Croft': 28,
 'Destroyed': 29,
 'Devonshire': 30,
 'Don': 31,
 'Dubarry': 32,
 'Emperors': 33,
 'Florence': 34,
 'For': 35,
 'Gallery': 36,
 'Gideon': 37,
 'Gisburn': 38,
 'Gisburns': 39,
 'Grafton': 40,
 'Greek': 41,
 'Grindle': 42,
 'Grindles': 43,
 'HAD': 44,
 'Had': 45,
 'Hang': 46,
 'Has': 47,
 'He': 48,
 'Her': 49,
 'Hermia': 50,
 'His': 51,
 'How': 52,
 'I': 53,
 'If': 54,
 'In': 55,
 'It': 56,
 'Jack': 57,
 'Jove': 58,
 'Just': 59,
 'Lord': 60,
 'Made': 61,
 'Miss': 62,
 'Money': 63,
 'Monte': 64,
 'Moon-dancers': 65,
 'Mr': 66,
 'Mrs': 67,
 'My': 68,
 'Never': 69,
 'No': 70,
 'Now': 71,
 'Nutley': 72,
 'Of': 73,
 'Oh': 74,
 'On': 75,
 'Once': 76,
 'Only': 77,
 '

In [9]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [word for word in preprocessed if word.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [10]:
tokenizer = SimpleTokenizerV1(vocab)

In [11]:
text = """ It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""

In [12]:
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [13]:
tokenizer.decode(tokenizer.encode(text))

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [14]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
len(vocab)

1132

In [15]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [word for word in preprocessed if word.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [16]:
tokenizer = SimpleTokenizerV2(vocab)
text = "Hello, my name is Hrishi sdhireby7n8qvf xkjovm90r4<|endoftext|>"
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, my <|unk|> is <|unk|> <|unk|> <|unk|>'

In [17]:
tokenizer = tiktoken.get_encoding("gpt2")
tokenizer.decode(tokenizer.encode(text, allowed_special= {"<|endoftext|>"}))

'Hello, my name is Hrishi sdhireby7n8qvf xkjovm90r4<|endoftext|>'

In [18]:
with open("../data/the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
len(enc_text)

5145

In [19]:
enc_sample = enc_text[50:]
len(enc_sample)

5095

In [20]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [21]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "----->", tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


In [22]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [23]:
def create_dataloader_v1(txt, batch_size = 4, max_length = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader

In [24]:
dataloader = create_dataloader_v1(raw_text, batch_size = 1, max_length = 4, stride = 4, shuffle = False)

dataiter = iter(dataloader)
first_batch = next(dataiter)
first_batch

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

In [25]:
second_batch = next(dataiter)
second_batch

[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]

In [26]:
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = 4, stride = 4, shuffle = False)

dataiter = iter(dataloader)
inputs, targets = next(dataiter)
print(f"Inputs: {inputs}")
print(f"Targets: {targets}")

Inputs: tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets: tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [27]:
input_ids = torch.tensor([2, 3, 5, 1])

In [28]:
output_dim = 256

embedding_layer = torch.nn.Embedding(tokenizer.n_vocab, output_dim)

In [29]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.7136, -0.4338, -0.1094,  ..., -0.2323, -1.1017, -0.0208],
        [-0.2875,  0.7578, -0.9178,  ..., -1.1492,  1.1453, -0.7568],
        [ 0.6445, -0.1564, -0.3280,  ...,  1.2031, -0.9586,  1.2067],
        ...,
        [ 0.0192, -0.2825,  0.6515,  ...,  0.8889, -0.0974, -0.3971],
        [-0.1553, -0.9145,  1.4608,  ...,  0.0653,  0.9837,  0.8641],
        [-1.2479,  0.1346,  0.4242,  ...,  0.9357,  0.4700, -0.9194]],
       requires_grad=True)


In [30]:
embedding_layer(input_ids)

tensor([[ 0.6445, -0.1564, -0.3280,  ...,  1.2031, -0.9586,  1.2067],
        [ 0.2761, -1.0844,  1.8727,  ..., -1.0283, -1.2302, -1.5648],
        [ 2.2184,  0.4345,  0.2636,  ...,  1.5396, -0.4744,  0.1283],
        [-0.2875,  0.7578, -0.9178,  ..., -1.1492,  1.1453, -0.7568]],
       grad_fn=<EmbeddingBackward0>)

In [31]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = max_length, stride = max_length, shuffle = False)

dataiter = iter(dataloader)
inputs, targets = next(dataiter)

In [32]:
token_embeddings = embedding_layer(inputs)

In [33]:
token_embeddings.shape

torch.Size([8, 4, 256])

In [34]:
token_embeddings[0, 0]

tensor([-1.4232e+00,  1.6929e+00, -5.1201e-01, -6.2081e-01,  6.8535e-01,
        -8.3855e-01, -2.0455e+00, -1.8620e+00, -1.6667e+00, -4.6455e-01,
         3.3495e+00, -2.3660e-01, -7.2925e-01,  1.3038e-01,  6.8886e-01,
        -1.9370e-02, -8.3490e-01, -1.7578e+00,  9.9186e-01,  5.3972e-01,
        -8.2576e-01,  1.0565e+00, -3.9027e-01, -1.8472e+00, -1.0531e+00,
        -7.2722e-01,  1.1802e+00,  1.1757e+00,  6.1428e-03,  1.5180e+00,
         8.2863e-01, -6.9013e-01,  1.4875e+00, -5.7411e-01, -1.1284e+00,
         1.0447e+00, -1.0043e-01, -7.5121e-01, -4.6995e-02, -1.5908e-01,
         6.1809e-01, -2.4256e+00,  5.6325e-01,  1.5869e-01,  4.0308e-01,
        -6.1913e-01, -6.4794e-01,  1.2881e+00, -1.2870e+00, -5.3166e-01,
        -1.0581e+00,  3.1468e-01, -8.5540e-01,  6.4850e-01,  8.4457e-02,
        -1.3024e+00,  1.1640e+00, -2.4470e-01,  1.7656e-03,  6.0374e-01,
        -2.2562e-01,  6.3414e-01,  4.6199e-02, -4.0756e-01,  9.9898e-02,
        -4.9370e-01, -1.9714e+00, -5.0296e-01,  4.2

In [35]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [36]:
torch.arange(max_length)

tensor([0, 1, 2, 3])

In [37]:
pos_embedding_layer.weight

Parameter containing:
tensor([[-0.0724,  1.8673, -0.4871,  ...,  0.3375,  0.3493, -0.0376],
        [-0.1424,  0.5137,  1.7663,  ..., -0.2311, -1.3547,  0.8542],
        [-2.3813, -0.5838, -0.2141,  ...,  1.4277, -0.1529, -1.0750],
        [ 0.6347,  0.1482, -0.0528,  ..., -2.0833,  0.8007, -0.0256]],
       requires_grad=True)

In [38]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
pos_embeddings.shape

torch.Size([4, 256])

In [39]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
